# 1.Library

In [8]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# 2.Data Loader

In [9]:
def load_mnist_binary_data(data_dir, honest_range=100, poison_range=100, img_size=28,select='Honest+Poisoned'):
    X = []
    y = []
    def load_honest(X, y):
        for i in range(honest_range):
            folder = os.path.join(data_dir, f"honest_{i}")
            if not os.path.exists(folder):
                continue
            for label in os.listdir(folder):
                label_folder = os.path.join(folder, label)
                if not os.path.isdir(label_folder):
                    continue
                for fname in os.listdir(label_folder):
                    if fname.endswith('.png') or fname.endswith('.jpg'):
                        img = Image.open(os.path.join(label_folder, fname)).convert('L').resize((img_size, img_size))
                        X.append(np.array(img).flatten() / 255.0)
                        y.append(label)
        return X, y
    def load_poisoned(X, y):
        for i in range(poison_range):
            folder = os.path.join(data_dir, f"poison_{i}")
            if not os.path.exists(folder):
                continue
            for label in os.listdir(folder):
                label_folder = os.path.join(folder, label)
                if not os.path.isdir(label_folder):
                    continue
                for fname in os.listdir(label_folder):
                    if fname.endswith('.png') or fname.endswith('.jpg'):
                        img = Image.open(os.path.join(label_folder, fname)).convert('L').resize((img_size, img_size))
                        X.append(np.array(img).flatten() / 255.0)
                        y.append(label)
        return X, y
    
    if select == 'Honest+Poisoned':
        X, y = load_honest(X, y)
        X, y = load_poisoned(X, y)
    elif select == 'Poisoned+Honest':
        X, y = load_poisoned(X, y)
        X, y = load_honest(X, y)
    elif select == 'Honest':
        X, y = load_honest(X, y)
    elif select == 'Poisoned':
        X, y = load_poisoned(X, y)

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    return X, y

def load_mnist_binary_test_data(test_dir, img_size=28):
    X = []
    y = []
    if not os.path.exists(test_dir):
        return X, y
    for label in ['0', '1']:
        label_folder = os.path.join(test_dir, label)
        if not os.path.isdir(label_folder):
            continue
        for fname in os.listdir(label_folder):
            if fname.endswith('.png') or fname.endswith('.jpg'):
                img = Image.open(os.path.join(label_folder, fname)).convert('L').resize((img_size, img_size))
                X.append(np.array(img).flatten() / 255.0)
                y.append(label)
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    return X, y

# 3.Model

In [10]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 1)
    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x

# 4.Loss

In [11]:
def binary_cross_entropy(pred, target):
    eps = 1e-7
    pred = torch.clamp(pred, eps, 1 - eps)
    return -(target * torch.log(pred) + (1 - target) * torch.log(1 - pred)).mean()


# 5.Configuration

In [12]:
N_HONEST = 100
N_POISONED = 100
IMG_SIZE = 28
LEARNING_RATE = 0.001
EPOCHS = 100
BATCH_SIZE = 32
N_CLASSES = 2

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

DATA_PATH = "../data/mnist_binary_poison/train"
TEST_PATH = "../data/mnist_binary_poison/test"

RESULT_PATH = "./results"


# 6.Main

In [13]:
poison_percent = [0, 10, 20, 30, 40, 50]
poison_n_list = []
for p in range(0,N_POISONED+1):
    percent = int((p / (p+N_HONEST)) * 100)
    if percent in poison_percent:
        poison_percent.remove(percent)
        print(f"Adding poison n: {p} for percent: {percent}")
        poison_n_list.append(p)
print("Poison n list:", poison_n_list)

Adding poison n: 0 for percent: 0
Adding poison n: 12 for percent: 10
Adding poison n: 25 for percent: 20
Adding poison n: 43 for percent: 30
Adding poison n: 67 for percent: 40
Adding poison n: 100 for percent: 50
Poison n list: [0, 12, 25, 43, 67, 100]


In [14]:

BATCH_SIZE_list = [BATCH_SIZE]
# temp_bs = BATCH_SIZE
# while temp_bs >= 1:
#     BATCH_SIZE_list.append(temp_bs)
#     temp_bs = temp_bs // 2

for rev in range(3):

    model = MNISTNet().to(device)
    for experiment_bs in BATCH_SIZE_list:

        name_save_path = f"CL_ModelN1L1_Batchsize{experiment_bs}_rev{rev}"
        
        for i_poisoned in poison_n_list:
            percent_poisoned = int((i_poisoned / (i_poisoned + N_HONEST)) * 100)

            # create directory
            NAME_SAVE_update_PATH = f"poisoned_{percent_poisoned}percent"
            save_path = os.path.join(RESULT_PATH,name_save_path,NAME_SAVE_update_PATH)
            # Remove existing directory if it exists
            if os.path.exists(save_path):
                shutil.rmtree(save_path)
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            save_all_path = os.path.join(RESULT_PATH,name_save_path,'all')
            os.makedirs(save_all_path, exist_ok=True)
            
            # Load data
            X_train, y_train = load_mnist_binary_data( DATA_PATH, honest_range=N_HONEST, 
                                                    poison_range=i_poisoned, img_size=IMG_SIZE, 
                                                    select='Honest+Poisoned')
            X_test, y_test = load_mnist_binary_test_data( TEST_PATH, img_size=IMG_SIZE)
            # print(f"Loaded train: X={X_train.shape}, y={y_train.shape}")
            # print(f"Loaded test: X={X_test.shape}, y={y_test.shape}")

            if X_test.shape[0] == 0:
                raise ValueError("No valid test images found in the test directory!")

            X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
            y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
            X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
            y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

            torch.save(model, os.path.join(save_path,'model_init.pt'))
            optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

            records = []

            # Training loop
            for epoch in range(EPOCHS):

                # Train
                model.train()
                
                #randomize the training data
                perm = torch.randperm(X_train_tensor.size(0))
                X_train_tensor = X_train_tensor[perm]
                y_train_tensor = y_train_tensor[perm]

                for start in range(0, X_train_tensor.size(0), experiment_bs):
                    end = start + experiment_bs
                    xb = X_train_tensor[start:end]
                    yb = y_train_tensor[start:end]
                    outputs = model(xb)
                    preds = (outputs >= 0.5).float()
                    acc_train = (preds == yb).float().mean().item()
                    loss_train = binary_cross_entropy(outputs, yb)
                    optimizer.zero_grad()
                    loss_train.backward()
                    optimizer.step()

                # Evaluate
                model.eval()
                with torch.no_grad():
                    outputs = model(X_test_tensor)
                    preds = (outputs >= 0.5).float()
                    loss_test = binary_cross_entropy(outputs, y_test_tensor)
                    acc_test = (preds == y_test_tensor).float().mean().item()
                records.append({'poison_percent': percent_poisoned, 'epoch': epoch, 
                                'loss_train': loss_train.item(), 'acc_train': acc_train, 
                                'loss_test': loss_test.item(), 'acc_test': acc_test})
                report_txt = f"poison_percent {percent_poisoned} "
                report_txt += f"epoch {epoch}: "
                report_txt += f"loss_train={loss_train.item():.4f}, "
                report_txt += f"acc_train={acc_train:.4f}, "
                report_txt += f"loss_test={loss_test.item():.4f}, "
                report_txt += f"acc_test={acc_test:.4f}"
                print(report_txt)

            torch.save(model, os.path.join(save_path,'model_last.pt'))

            # Save training log
            df = pd.DataFrame(records)
            save_name_path = os.path.join(save_path, f'{NAME_SAVE_update_PATH}.csv')
            df.to_csv(save_name_path, index=False)
            print(f"Training log saved to {save_name_path}")

            # Plot loss and accuracy
            plt.figure(figsize=(10,4))

            plt.subplot(1,2,1)
            plt.plot(df['epoch'], df['loss_test'], marker='o')
            plt.title('Test Loss')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.grid(True)
            plt.ylim(0, 1.1)
            # plt.legend()

            plt.subplot(1,2,2)
            plt.plot(df['epoch'], df['acc_test'], marker='o')
            plt.title('Test Accuracy')
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.grid(True)
            plt.ylim(0, 1.1)
            # plt.legend()

            plt.tight_layout()
            save_name_path = os.path.join(save_path, f'loss_accuracy.jpg')
            plt.savefig(save_name_path)

            save_name_path = os.path.join(save_all_path, f'{NAME_SAVE_update_PATH}_latest.jpg')
            plt.savefig(save_name_path)
            plt.close()


poison_percent 0 epoch 0: loss_train=0.0778, acc_train=1.0000, loss_test=0.0459, acc_test=0.9980
poison_percent 0 epoch 1: loss_train=0.0120, acc_train=1.0000, loss_test=0.0211, acc_test=0.9990
poison_percent 0 epoch 2: loss_train=0.0084, acc_train=1.0000, loss_test=0.0134, acc_test=0.9995
poison_percent 0 epoch 3: loss_train=0.0060, acc_train=1.0000, loss_test=0.0097, acc_test=0.9995
poison_percent 0 epoch 4: loss_train=0.0040, acc_train=1.0000, loss_test=0.0076, acc_test=0.9995
poison_percent 0 epoch 5: loss_train=0.0023, acc_train=1.0000, loss_test=0.0062, acc_test=0.9995
poison_percent 0 epoch 6: loss_train=0.0018, acc_train=1.0000, loss_test=0.0053, acc_test=0.9995
poison_percent 0 epoch 7: loss_train=0.0024, acc_train=1.0000, loss_test=0.0046, acc_test=0.9995
poison_percent 0 epoch 8: loss_train=0.0011, acc_train=1.0000, loss_test=0.0040, acc_test=0.9995
poison_percent 0 epoch 9: loss_train=0.0013, acc_train=1.0000, loss_test=0.0036, acc_test=0.9995
poison_percent 0 epoch 10: los